In [1]:
from mongodb_ju import collection_r, collection_c_jp, collection_c_w  # 몽고디비 연결
from slack import send_slack  # slack.py 모듈
from crawling_recruit import JobRecruit  # recruit 크롤링 모듈
import pandas as pd
%% writefile jobplanet_all.py
# -*- coding: utf-8 -*-


class jobplanet_all():

    def get_old_recruit_df(self):
        # 크롤링 전 mongodb에 저장된 데이터를 데이터프레임으로 저장
        # old_mongodb 를 df로 만들기(pandas가 다루기 편하기 때문!)
        old_recruit_df_jp = pd.DataFrame(list(collection_c_jp.find()))
        old_recruit_df_w = pd.DataFrame(list(collection_c_w.find()))
        # pickle로 저장
        old_recruit_df_jp.to_pickle("job_search/old_recruit_df_jp.pkl")
        old_recruit_df_w.to_pickle("job_search/old_recruit_df_w.pkl")

    def recruit_crawling(self):
        # ju: juhyun's mongodb에 저장
        jpr = JobRecruit()
        recruit = jpr.save_to_mongodb_j('ju')
        wr = JobRecruit()
        wrecruit = wr.save_to_mongodb_w('ju')

    def review_crawling(self):
        # 회사이름검색
        C_jp_company_name_distinct = collection_c_jp.distinct("company_name")
        C_w_company_name_distinct = collection_c_w.distinct("company_name")
        R_company_name_distinct = collection_r.distinct("company_name")

        # recruit의 회사이름들 중 review에는 없는 회사이름 검색
        company_name_recruit_only_jp = list(
            set(C_jp_company_name_distinct)-set(R_company_name_distinct))
        company_name_recruit_only_w = list(
            set(C_w_company_name_distinct)-set(R_company_name_distinct))

        # companay_name_recruit_only : jobplanet+wanted 합친후 중복제거
        company_name_recruit_only = company_name_recruit_only_jp+company_name_recruit_only_w
        company_name_recruit_only = list(set(company_name_recruit_only))

        self.write_txt(company_name_recruit_only, 'job_search/only.txt', ' ')

    # company_name_recruit_only를 텍스트 파일로 저장하는 함수
    def write_txt(self, list, fname, sep):
        file = open(fname, 'w')
        vstr = ''

        for a in list:
            vstr = vstr+str(a)+sep
        vstr = vstr.rstrip(sep)

        file.writelines(vstr)

        file.close()
        print('[파일저장완료]')

    # new_minus_old : df의 차집합 구하기
    def new_minus_old(self, old, new):
        # old와 new를 행으로 합치기
        old_new_concat = pd.concat([old, new], keys=list(new.columns), axis=1)
        # old의 multiindex만 뽑아내기
        list(old_new_concat.columns[:int(len(old_new_concat.columns)/2)])
        # old의 모든 행이 NAN인 행 뽑아내기
        null_df = old_new_concat[list(
            old_new_concat.columns[:int(len(old_new_concat.columns)/2)])].isnull()
        # null_df의 Multiindex 없애기
        null_df.columns = null_df.columns.droplevel()
        # new에서 null_df의 값을 반환
        # new_minus_old : 우리가 원하는 값! new에만 있는 값!
        new_minus_old = new[null_df].dropna(how='all').drop_duplicates()
        return new_minus_old

    def new_mongo_db(self, which_old_recruit_df):
        old_recruit_df = pd.read_pickle(
            "job_search/{}.pkl".format(which_old_recruit_df))

        if which_old_recruit_df == 'old_recruit_df_jp':
            # old+new된 newmongodb
            new_recruit_df = pd.DataFrame(list(collection_c_jp.find()))

        elif which_old_recruit_df == 'old_recruit_df_w':
            # old+new된 newmongodb
            new_recruit_df = pd.DataFrame(list(collection_c_w.find()))

        new_recruit_df = new_recruit_df.drop('_id', axis=1)
        # new_mongodb_df의 duplicate를 drop 하여 다시 저장
        new_recruit_df = new_recruit_df.drop_duplicates()
        new_recruit_df = new_recruit_df.reset_index(drop=True)
        #result_mongodb_df= new_mongodb_df-old_mongodb_df
        result_recruit_df = self.new_minus_old(old_recruit_df, new_recruit_df)
        result_recruit_df = result_recruit_df.reset_index(drop=True)
        return result_recruit_df

    def slack(self, result_recruit_df, from_which):
        # company_names,titles,flask_urls->var_zip
        company_names = [result_recruit_df.loc[i].company_name for i in range(
            len(result_recruit_df))]
        titles = [result_recruit_df.loc[i].title for i in range(
            len(result_recruit_df))]
        if from_which == 'Jobplanet':
            flask_urls = ['http://15.164.118.245/jobplanet/{}/{}'.format(
                company_name, title) for company_name, title in list(zip(company_names, titles))]
        elif from_which == 'Wanted':
            flask_urls = ['http://15.164.118.245/wanted/{}/{}'.format(
                company_name, title) for company_name, title in list(zip(company_names, titles))]
        var_zip = list(zip(company_names, titles, flask_urls))

        # message
        message = ['<{} :{}> {} 에서 새로운 채용공고가 떴습니다'.format(from_which, i, var_zip[i][0])+(' '*5) +
                   '*공고제목:{}'.format(var_zip[i][1])+(' '*5) +
                   '*공고와 회사리뷰 자세히 보기:{}'.format(var_zip[i][2]) for i in range(len(var_zip))]
        # send_slack
        for msg in message:
            send_slack('{}'.format(msg))

    # 크롤링 전 df 저장 -> recruit 크롤링 -> reveiw 크롤링 전체

    def to_review_crawling(self):
        self.get_old_recruit_df()  # 크롤링 이전, 몽고db의 테이블을 객체로 저장
        self.recruit_crawling()  # 공고 크롤링
        self.review_crawling()  # 리뷰 크롤링

    def to_slack(self):
        # jobplanet
        # 크롤링 이전의 몽고db 객체와 크롤링 이후의 몽고db 데이터프레임을 비교함
        result_recruit_df_jp = self.new_mongo_db('old_recruit_df_jp')
        # 크롤링 이후의 몽고 db에 새롭게 추가된 내용을 slack 메시지로 전송
        self.slack(result_recruit_df_jp, 'Jobplanet')
        # wanted
        result_recruit_df_w = self.new_mongo_db('old_recruit_df_w')
        self.slack(result_recruit_df_w, 'Wanted')

Overwriting jobplanet_all.py


### crontab 실행 모듈

In [3]:
from jobplanet_all import *
%% writefile to_review_crawling.py
# -*- coding: utf-8 -*-
jobplanet = jobplanet_all()
jobplanet.to_review_crawling()

Overwriting to_review_crawling.py


In [ ]:
!./run.sh

In [4]:
from jobplanet_all import *
%% writefile to_slack.py
jobplanet = jobplanet_all()
jobplanet.to_slack()
print("slack 메세지 보내기 완료")

Overwriting to_slack.py
